In [25]:
import random as rnd
import pymorphy2 as pmh

import yargy
from yargy.tokenizer import MorphTokenizer
from yargy import Parser, rule, and_, or_
from yargy.predicates import gram, dictionary, custom, true
from yargy.pipelines import morph_pipeline

import itertools as it
import typing as t
from overrides import overrides

In [26]:
class SimpleTextGenerator:
    """Simple Text Generator

    Simple text generator builds texts just using strings as parts. Not support any formatting or 
    variants.
    """
    
    def __init__(self, parts: t.List[str]) -> None:
        """Constructor

        :param parts: 
        """

        self.__parts = parts

    def generate(
        self,
        joiner: t.Callable[[t.Iterable[str]], str] = ' '.join,
        filter_: t.Callable[[t.Iterable[str]], bool] = lambda combination: True,
    ) -> t.Iterable[str]:
        combinations = it.chain.from_iterable((
            it.combinations(self.__parts, combination_size)
            for combination_size in range(1, len(self.__parts) + 1)
        ))

        combinations_with_permutations = it.chain.from_iterable((
            it.permutations(combination)
            for combination in combinations
        ))

        return (
            joiner(combination) 
            for combination in combinations_with_permutations 
            if filter_(combination)
        )
class TextGenerator:    
    def __init__(self, components: t.List[t.List[str]]) -> None:
        self.__components = components

    def generate(
        self,
        joiner: t.Callable[[t.Iterable[str]], str] = ' '.join,
        filter_: t.Callable[[t.Iterable[str]], bool] = lambda combination: True,) -> t.Iterable[str]:
        return it.chain.from_iterable((
            SimpleTextGenerator(parts).generate(joiner, filter_)
            for parts in it.product(*self.__components)
        ))

In [27]:
gen = TextGenerator([
    ["купить"],
    ["велик", "самокат", "велосипед", "байк", "скейт", "скейтбоард"],
    ["от 20к", "от 13 тыс", "от 14 тысяч"],
    ["до 40к", "до 56 тысяч"],
    ["кэшбек", "с кэшбеком", "20% кэшбек", "с кэшбеком 3 процента"],
    ["15%", "5к"]
])
texts = list(gen.generate())

In [28]:
prev = [
    "купить",
    "где купить",
    ""
]
goods = [
    "велосипед",
    "велик",
    "байк",
    "скейт",
    "скейтбоард"
]
credits = [
    "в кредит",
    "в рассрочку",
    "под 12 процентов",
    "14%"
]
morph = pmh.MorphAnalyzer()

In [29]:
def getRequire():
    return prev[0] + " " + goods[0] + " стоит меньше трёх тыс руб" + " " + credits[0]

In [30]:
def Lemma(string):
    newString = ""
    for word in string.split(" "):
        newString += morph.parse(word)[0].normal_form + " "
    return newString[:-1]

In [31]:
Lemma(getRequire())

'купить велосипед стоить маленький три тысяча рубль в кредит'

In [32]:
def is_number(string):
    for c in string:
        if((ord(c) < 48 or ord(c) > 57) and not (c == "." or c == ",")):
            return False
    return True
is_number_ = custom(is_number)
NUMBER_RULE = rule(
    or_(
        gram("NUMR"),
        is_number_
    ),
    morph_pipeline([
        ",",
        "."
    ]).optional(),
    or_(
        gram("NUMR"),
        is_number_
    ).optional()
)

In [34]:
MONEY_PIPE = morph_pipeline([
        "тыс",
        "к",
        "k",
        "м",
        "руб",
        "рублей",
        "тысяч"
    ])

In [35]:
PRICE_FROM = rule(
    morph_pipeline([
        "от"
    ]),
    NUMBER_RULE,
    MONEY_PIPE.optional()
)
PRICE_TO = rule(
    morph_pipeline([
        "до"
    ]),
    NUMBER_RULE,
    MONEY_PIPE.optional()
)
PRICE_VALUE = rule(
    NUMBER_RULE,
    not_(
        dictionary({
            "%",
            "процент",
            "процентов"
        })
    ),
    MONEY_PIPE.optional()
)

In [36]:
SPORT_ITEM = rule(
    and_(
        gram("ADJF"),
        not_(
            gram("PREP")
        )
    ).optional().repeatable(),
    gram("NOUN"),
    and_(
        gram("ADJF"),
        not_(
            gram("PREP")
        )
    ).optional().repeatable()
)

In [48]:
parser = Parser(SPORT_ITEM)
for match in parser.findall("купить велосипед с держателем для воды недорого кэшбек 5%"):
    print([_.value for _ in match.tokens])

['велосипед']
['с']
['держателем']
['воды']
['кэшбек']


In [37]:
MONEY_PERCENT = rule(
    or_(
    rule(
        morph_pipeline([
            "процент",
            "%"
        ]).optional(),
        MONEY_PIPE
        ),
    rule(
        morph_pipeline([
            "процент",
            "%"
        ]),
        MONEY_PIPE.optional()
    )
    )
)
CASHBACK_VALUE = rule(
    NUMBER_RULE,
    MONEY_PERCENT.optional()
)
CASHBACK = rule(
    NUMBER_RULE.optional().repeatable(),
    MONEY_PERCENT.optional(),
    morph_pipeline([
        "кэшбек",
        "кэшбэк",
        "cb",
        "кб",
        "кэш"
    ]),
    dictionary({
        "от"
    }).optional(),
    NUMBER_RULE.optional().repeatable(),
    MONEY_PERCENT.optional()
)
PERCENT_RULE = rule(
    NUMBER_RULE,
    morph_pipeline([
        "%",
        "процент"
    ])
)
text = "купить горный велсипед с кэшбеком 5.2 % за 50к"
p = Parser(PERCENT_RULE)
p.findall(text)
for match in p.findall(text):
    print([_.value for _ in match.tokens])

['5', '.', '2', '%']


In [38]:
cbr = Parser(CASHBACK)
txt = "купить горный велик в кредит 10% кэшбек до 20к"
tokens = cbr.findall(txt)
for match in tokens:
    for _ in match.tokens:
        txt = txt.replace(_.value, "")
        print(_.value, " ", _.span)
txt

10   [29, 31)
%   [31, 32)
кэшбек   [33, 39)


'купить горный велик в кредит   до 20к'

Строка должна поступать на вход с пробелами по краям: " купить велосипед с кэшбеком 5 % "

In [39]:
tokenizer = MorphTokenizer()
string = " "
for i in t(txt):
    print(i.forms[0].grams.__dict__)
string

TypeError: 'module' object is not callable

In [69]:
MEANING = rule(
    not_(
        or_(
            or_(
                gram("INFN"),
                gram("VERB")
            ),
            or_(
                or_(
                    gram("PREP"), gram("CONJ")
                ),
                or_(
                    gram("PRCL"), gram("ADVB")
                )
            )
        )
    )
)
ATTRIBUTE = rule(
    MEANING.optional().repeatable(),
    gram("NOUN").repeatable()
)

In [73]:
import sys
sys.executable

'C:\\Program Files (x86)\\Microsoft Visual Studio\\Shared\\Anaconda3_64\\python.exe'

In [87]:
import pymorphy2 as pmh
par = pmh.MorphAnalyzer()
class SlotFilling:
    def __init__(self):
        self.dict = dict()
        self.dict["goods"] = goods
        self.price_rules = [PRICE_FROM, PRICE_TO]
        self.tokenizer = MorphTokenizer()
    def Preprocess(self, string):
        processed_string = " "
        for token in self.tokenizer(string):
            processed_string += token.value + " "
        return processed_string
    def Parsing(self, string):
        words = string.split(" ")
        parsed = dict()
       
        #FIND CASHBACK
        parser = Parser(CASHBACK)
        cashback_tokens = parser.findall(string)
        cashback = ""
        #пока тренируемся на том, чnо кэшбек только на один товар
        for match in cashback_tokens:
            cashback += ' '.join([_.value for _ in match.tokens])
            for token in match.tokens:
                string = string.replace(" " + token.value + " ", " ")
        #вытаскиваем значения с размерностями:
        parser = Parser(CASHBACK_VALUE)
        cashback_tokens = parser.findall(cashback)
        cashback = ""
        for match in cashback_tokens:
            cashback += ' '.join([_.value for _ in match.tokens])
        #проверяем просто на вхождение процентов (т.к. пока мы рассрочку не учитываем)
        if(cashback == ""):
            parser = Parser(PERCENT_RULE)
            cashback_tokens = parser.findall(cashback)
            for match in cashback_tokens:
                cashback += ' '.join([_.value for _ in match.tokens])
        parsed['Cashback'] = cashback.replace(" ", "")
        #find  price
        parsed['Price'] = {"From" : "NaN", "To": "NaN"}
        is_value = 0
        price_keys_list = list(parsed['Price'].keys())
        for i in range(2):
            parser = Parser(self.price_rules[i])
            price_tokens = parser.findall(string)
            for match in price_tokens:
                is_value += 1
                parsed['Price'][price_keys_list[i]] = ' '.join([_.value for _ in match.tokens])
                for token in match.tokens:
                    string = string.replace(token.value + " ", " ")
        if (is_value == 0):
            parser = Parser(PRICE_VALUE)
            price_tokens = parser.findall(string)
            price = ""
            for match in price_tokens:
                price = ' '.join([_.value for _ in match.tokens])
                parsed['Price']["From"] = parsed['Price']["To"] = price
                for token in match.tokens:
                    string = string.replace(token.value + " ", "")
        
        print(string)
        parser = Parser(ATTRIBUTE)
        attr_tokens = parser.findall(string)
        attr = ""
        parsed['Attributes'] = "NaN"
        for match in attr_tokens:
                attr = ' '.join([_.value for _ in match.tokens])
                parsed['Attributes'] = attr
        for word in words:
            #find Item
            if(par.parse(word)[0].normal_form in self.dict['goods']):
                parsed['Item'] = word
                #while True:
                #    pass
        
        return parsed
    def Parse(self, string):
        return self.Parsing(self.Preprocess(string))

In [88]:
SF = SlotFilling()
text = "купить горный велосипед за 50к"
SF.Parse(text)

 купить горный велосипед за 


{'Attributes': 'горный велосипед',
 'Cashback': '',
 'Item': 'велосипед',
 'Price': {'From': '50 к', 'To': '50 к'}}

In [43]:
SF = SlotFilling()
labeled = []

for _ in range(10):
    text = texts[rnd.randint(10000, len(texts))] 
    print(text)
    print(SF.Parse(text))
#labeled = [SF.Parse(text) for text in texts[:10]]

скейтбоард 15% купить до 56 тысяч от 14 тысяч
 скейтбоард 15 % купить 
{'Cashback': '', 'Price': {'From': 'от 14 тысяч', 'To': 'до 56 тысяч'}, 'Attributes': 'скейтбоард', 'Item': 'скейтбоард'}
до 56 тысяч кэшбек байк от 13 тыс 15% купить
 до байк 15 % купить 
{'Cashback': '56 тысяч', 'Price': {'From': 'от 13 тыс', 'To': 'NaN'}, 'Attributes': 'байк', 'Item': 'байк'}
от 13 тыс до 40к 5к купить
 5 купить 
{'Cashback': '', 'Price': {'From': 'от 13 тыс', 'To': 'до 40 к'}, 'Attributes': 'NaN'}
до 56 тысяч байк купить
 байк купить 
{'Cashback': '', 'Price': {'From': 'NaN', 'To': 'до 56 тысяч'}, 'Attributes': 'байк', 'Item': 'байк'}
с кэшбеком 3 процента 15% купить до 56 тысяч скейт от 14 тысяч
 с 15 % купить скейт 
{'Cashback': '3 процента', 'Price': {'From': 'от 14 тысяч', 'To': 'до 56'}, 'Attributes': 'скейт', 'Item': 'скейт'}
15% велик 20% кэшбек до 40к от 13 тыс
 15 велик 
{'Cashback': '20 %', 'Price': {'From': 'от 13 тыс', 'To': 'до 40 к'}, 'Attributes': 'велик', 'Item': 'велик'}
от 14 т

In [23]:
texts[10000:10010]

['от 20к 5к купить велик',
 'от 20к 5к велик купить',
 '5к купить велик от 20к',
 '5к купить от 20к велик',
 '5к велик купить от 20к',
 '5к велик от 20к купить',
 '5к от 20к купить велик',
 '5к от 20к велик купить',
 'купить велик до 56 тысяч кэшбек',
 'купить велик кэшбек до 56 тысяч']

***Show PRICE rule:***

In [251]:
text = "купить велосипед от 10 50 тыс"
parse = Parser(PRICE_FROM)
for match in parse.findall(text):
    print("12312")
    print([_.value for _ in match.tokens])

In [149]:
Installment = fact(
    "Installment",
    ["Bool", "Time", "Percent"]
)
Require = fact(
    "Require",
    ["Item", "Price", "Installment"]
)

In [150]:
INSTALLMENT_TIME = rule(
    or_(
        gram("NUMR"),
        is_number_
    ),
    morph_pipeline([
        "год",
        "месяц",
        "день",
        "лет"
    ])
)
INSTALLMENT_PERCENT = rule(
    or_(
        gram("NUMR"),
        is_number_
    ),
    morph_pipeline([
        "процент",
        "%"
    ])
)
INSTALLMENT = rule(
    INSTALLMENT_TIME.interpretation(
        Installment.Time.inflected()
    ),
    INSTALLMENT_PERCENT.interpretation(
        Installment.Percent.inflected()
    )
).interpretation(Installment)

In [151]:
parser = Parser(INSTALLMENT)

In [152]:
text = getRequire()
text = "купить велосипед меньше 15 тыс руб 12 месяц 14%"

In [153]:
from IPython.display import display
for match in parser.findall(text):
    display(match.fact)

Installment(Bool=None,
            Time='12 месяц',
            Percent='14%')

In [16]:
from yargy.tokenizer import MorphTokenizer

In [17]:
tokenizer = MorphTokenizer()
text = "12к"
for line in text.splitlines():
    print([_.value for _ in tokenizer(line)])

['12', 'к']
